# 🧹 Phase 2: Data Cleaning

## Objectives
- Handle missing values (Age, Cabin, Embarked)
- Address outliers in numerical features
- Remove or transform problematic columns
- Prepare clean dataset for feature engineering

## What You'll Learn
✅ Imputation strategies for missing data  
✅ Handling extreme outliers  
✅ Dropping irrelevant features  
✅ Data validation and verification  

---

**From EDA, we found:**
- Age: 20% missing → needs imputation
- Cabin: 77% missing → likely drop or extract deck letter
- Embarked: 2 missing → fill with mode
- Fare: has outliers → decide if we keep/cap them

## 1. Setup and Load Data

In [45]:
import numpy as np
import pandas as pd
import  matplotlib as plt
import seaborn as sns

In [46]:
# Load the dataset
# Your code here: read train.csv from ../data/
# Print shape and first few rows
df=pd.read_csv('C:/Users/moham/Downloads/Projects/Machine_learning/titanic-ml-pipeline/data/train.csv')
print(df.shape)
print(df.info())
print(df.describe().round(2))
print(df.head().round(2))


(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId  Survived  Pclass     Age   SibSp   Parch    Fare
count       891.00    891.00  891.00  714.00  891.00  891.00  891.00
mean        446.00      0.38    2.31   29.70    0.52    0.38   32.20
std         257.35    

In [47]:

missing_values=df.isnull().sum()
print(missing_values)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


---
## 2. Handle Missing Values

We'll tackle each column with missing data separately.

### 2.1 Age - Imputation Strategy

**Task:** Fill 177 missing Age values

**Goal**:
- Group by Pclass/Sex and use median per group (more sophisticated)


In [48]:

df['Age']=(df.groupby(['Sex','Pclass']))['Age'].transform(lambda x: x .fillna(x.mean()))


### 2.2 Cabin - Drop or Extract?

**Problem:** 687 missing values (77%)

**Options:**
1. Drop entire column (too much missing data)
2. Extract deck letter (first character: A, B, C, etc.) and create new feature
3. Create binary feature: HasCabin (0 or 1)

**Your choice!**

In [49]:

df['HasCabin']=df['Cabin'].notna().astype(int)
df.drop('Cabin',axis=1,inplace=True)


### 2.3 Embarked - Fill with Mode

**Problem:** Only 2 missing values

**Solution:** Fill with the most common port (mode)

In [50]:

df['Embarked'].fillna(df['Embarked'].mode()[0],inplace=True)


C:\Users\moham\AppData\Local\Temp\ipykernel_32660\4031226428.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0],inplace=True)


---
## 3. Handle Outliers

From EDA, we saw Fare has many high outliers.

### 3.1 Fare Outliers

**Options:**
1. Keep them (they're legitimate high prices)
2. Cap them at a reasonable value (e.g., 95th percentile)
3. Log transform to reduce skewness

**Recommendation:** Keep them for now (they contain real information about wealthy passengers)

In [51]:
df['Fare'].describe()


count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [52]:
cap_fare=df["Fare"].quantile(0.95)
df['Fare']=df['Fare'].clip(upper=cap_fare)

---
## 4. Drop Unnecessary Columns

Some columns won't help our model:
- **PassengerId**: Just an ID, no predictive value
- **Name**: Too unique (we'll extract Title in Phase 3)
- **Ticket**: Too many unique values, no clear pattern

In [53]:

df.drop('PassengerId',axis=1,inplace=True)
df.drop('Name',axis=1,inplace=True)
df.drop('Ticket',axis=1,inplace=True)
print(df.columns.tolist())

['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'HasCabin']


---
## 5. Verify Cleaning

Let's confirm all cleaning steps worked correctly.

In [54]:
# Check for any remaining missing values
print(df.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
HasCabin    0
dtype: int64


In [55]:
# Check dataset shape
print(df.shape)

(891, 9)


In [56]:
print(df.head())

   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked  HasCabin
0         0       3    male  22.0      1      0   7.2500        S         0
1         1       1  female  38.0      1      0  71.2833        C         1
2         1       3  female  26.0      0      0   7.9250        S         0
3         1       1  female  35.0      1      0  53.1000        S         1
4         0       3    male  35.0      0      0   8.0500        S         0


In [57]:
print(df.dtypes)

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
HasCabin      int64
dtype: object


---
## 6. Save Cleaned Data

Save the cleaned dataset for Phase 3 (Feature Engineering)

In [58]:
# Save cleaned data
df.to_csv('../data/train_cleaned.csv',index=False)
print(" Cleaned data saved to ../data/train_cleaned.csv")


 Cleaned data saved to ../data/train_cleaned.csv


---
## 7. Summary

**Cleaning Steps Completed:**

✅ Age: Imputed 177 missing values with median  
✅ Cabin: Dropped (77% missing)--> added HasCabin (0/1) instead

✅ Embarked: Filled 2 missing values with mode  
✅ Fare: capped outliers to 95%  
✅ Dropped: PassengerId, Name, Ticket  

**Result:** Clean dataset ready for feature engineering!

**Next Steps:**
- Phase 3: Feature engineering (extract titles, create family size, etc.)
- Phase 4: Model training